**Classifying CIFAR-10 **

Loading the datasets of tensorflow

In [1]:
!pip install tensorflow_datasets

Importing the datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [3]:
assert(tf.test.gpu_device_name())

tf.keras.backend.clear_session()
#XLA is disabled
tf.config.optimizer.set_jit(False) 

def load_data():
  result = tfds.load('cifar10', batch_size = -1)
  (x_train, y_train) = result['train']['image'],result['train']['label']
  (x_test, y_test) = result['test']['image'],result['test']['label']
  
  x_train = x_train.numpy().astype('float32') / 256
  x_test = x_test.numpy().astype('float32') / 256

  # Convert class vectors to binary class matrices.
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
  return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteP2TXC9/cifar10-train.tfrecord


...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteP2TXC9/cifar10-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [4]:
print(x_train)

[[[[0.55859375 0.375      0.2734375 ]
   [0.55078125 0.375      0.28125   ]
   [0.52734375 0.36328125 0.28125   ]
   ...
   [0.375      0.14453125 0.07421875]
   [0.41015625 0.1640625  0.0703125 ]
   [0.40625    0.1484375  0.078125  ]]

  [[0.5        0.3828125  0.359375  ]
   [0.5703125  0.4609375  0.4375    ]
   [0.6640625  0.56640625 0.5390625 ]
   ...
   [0.421875   0.17578125 0.1015625 ]
   [0.4375     0.171875   0.09375   ]
   [0.4375     0.16015625 0.0859375 ]]

  [[0.36328125 0.26953125 0.29296875]
   [0.4609375  0.375      0.39453125]
   [0.69921875 0.625      0.6328125 ]
   ...
   [0.5        0.265625   0.18359375]
   [0.48828125 0.23828125 0.1640625 ]
   [0.4765625  0.23046875 0.15234375]]

  ...

  [[0.73046875 0.5859375  0.48046875]
   [0.71875    0.578125   0.48046875]
   [0.69921875 0.5546875  0.47265625]
   ...
   [0.7734375  0.63671875 0.515625  ]
   [0.78515625 0.6484375  0.52734375]
   [0.80859375 0.6796875  0.55859375]]

  [[0.73046875 0.5859375  0.45703125]
   [0.7

In [7]:
print(y_train)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
print(x_test)

[[[[0.703125   0.7265625  0.73828125]
   [0.62890625 0.65234375 0.65625   ]
   [0.546875   0.5703125  0.5546875 ]
   ...
   [0.203125   0.21484375 0.15625   ]
   [0.19921875 0.21484375 0.15625   ]
   [0.19921875 0.2109375  0.15625   ]]

  [[0.671875   0.6953125  0.6875    ]
   [0.5859375  0.609375   0.59765625]
   [0.4609375  0.48828125 0.4609375 ]
   ...
   [0.17578125 0.18359375 0.12890625]
   [0.16796875 0.17578125 0.12109375]
   [0.171875   0.1875     0.1328125 ]]

  [[0.62890625 0.65625    0.625     ]
   [0.57421875 0.60546875 0.5703125 ]
   [0.46875    0.49609375 0.4609375 ]
   ...
   [0.1640625  0.1875     0.12890625]
   [0.16796875 0.1953125  0.1328125 ]
   [0.1796875  0.21875    0.15234375]]

  ...

  [[0.97265625 0.97265625 0.97265625]
   [0.98828125 0.98828125 0.98828125]
   [0.984375   0.984375   0.984375  ]
   ...
   [0.9921875  0.9921875  0.9921875 ]
   [0.99609375 0.99609375 0.99609375]
   [0.98046875 0.98046875 0.98046875]]

  [[0.97265625 0.97265625 0.97265625]
   [0.9

In [9]:
print(y_test)

[[0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [11]:
def generate_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', input_shape=x_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(128, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

In [12]:
def compile_model(model):
  opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  return model

model = compile_model(model)

def train_model(model, x_train, y_train, x_test, y_test, epochs=25):
  model.fit(x_train, y_train, batch_size=512, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

def warmup(model, x_train, y_train, x_test, y_test):
  # Warm up the JIT, we do not wish to measure the compilation time.
  initial_weights = model.get_weights()
  train_model(model, x_train, y_train, x_test, y_test, epochs=1)
  model.set_weights(initial_weights)

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

98/98 [==============================] - 31s 203ms/step - loss: 2.0255 - accuracy: 0.2613 - val_loss: 1.7800 - val_accuracy: 0.3745
Epoch 1/25
98/98 [==============================] - 18s 188ms/step - loss: 2.1589 - accuracy: 0.2032 - val_loss: 1.9333 - val_accuracy: 0.3117
Epoch 2/25
98/98 [==============================] - 18s 187ms/step - loss: 1.8370 - accuracy: 0.3393 - val_loss: 1.6725 - val_accuracy: 0.4145
Epoch 3/25
98/98 [==============================] - 18s 187ms/step - loss: 1.6831 - accuracy: 0.3972 - val_loss: 1.5802 - val_accuracy: 0.4390
Epoch 4/25
98/98 [==============================] - 18s 186ms/step - loss: 1.6011 - accuracy: 0.4254 - val_loss: 1.5283 - val_accuracy: 0.4541
Epoch 5/25
98/98 [==============================] - 18s 186ms/step - loss: 1.5282 - accuracy: 0.4526 - val_loss: 1.4465 - val_accuracy: 0.4933
Epoch 6/25
98/98 [==============================] - 18s 186ms/step - loss: 1.4641 - accuracy: 0.4779 - val_loss: 1.4257 - val_accuracy: 0.4978
Epoch 7/25

In [13]:

tf.keras.backend.clear_session()

#Enabling XLA
tf.config.optimizer.set_jit(True) 
model = compile_model(generate_model())
(x_train, y_train), (x_test, y_test) = load_data()

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

98/98 [==============================] - 24s 172ms/step - loss: 1.9881 - accuracy: 0.2758 - val_loss: 1.7810 - val_accuracy: 0.3702
Epoch 1/25
98/98 [==============================] - 21s 217ms/step - loss: 2.1211 - accuracy: 0.2129 - val_loss: 1.8436 - val_accuracy: 0.3525
Epoch 2/25
98/98 [==============================] - 16s 162ms/step - loss: 1.7980 - accuracy: 0.3530 - val_loss: 1.7317 - val_accuracy: 0.3838
Epoch 3/25
98/98 [==============================] - 16s 162ms/step - loss: 1.6675 - accuracy: 0.4026 - val_loss: 1.5553 - val_accuracy: 0.4431
Epoch 4/25
98/98 [==============================] - 16s 162ms/step - loss: 1.5891 - accuracy: 0.4294 - val_loss: 1.4981 - val_accuracy: 0.4632
Epoch 5/25
98/98 [==============================] - 16s 161ms/step - loss: 1.5198 - accuracy: 0.4533 - val_loss: 1.4327 - val_accuracy: 0.4878
Epoch 6/25
98/98 [==============================] - 16s 161ms/step - loss: 1.4587 - accuracy: 0.4784 - val_loss: 1.3845 - val_accuracy: 0.5099
Epoch 7/25